# Predicting absolute permeability

The example explains absolute permeabilty calculations on a cubic network. Note that permeability calcualtion for an extracted network from PoreSpy follows similar steps in assigning phase, algorithm and calculating permeability.

In [1]:
import numpy as np
import openpnm as op
%config InlineBackend.figure_formats = ['svg']
np.random.seed(10)
%matplotlib inline
np.set_printoptions(precision=5)

### Create a random cubic network

In [2]:
pn = op.network.Cubic(shape=[15, 15, 15], spacing=1e-6)
pn.add_model_collection(op.models.collections.geometry.spheres_and_cylinders)
pn.regenerate_models()

### Create phase object

It is assumed that a generic phase flowing through the porous medium. As absolute permeability is the porous medium property and not the fluid property, any other fluid asumming a viscosity value can be used as the phase.

In [3]:
phase = op.phase.Phase(network=pn)
phase['pore.viscosity']=1
phase.add_model_collection(op.models.collections.physics.basic)
phase.regenerate_models()

### Apply Stokes flow

To calculate permeability in x direction, a constant pressure boundary condition is applied on the left and right side of the network. Note that a similar procedure can be followed to find the permeability in y and z directions.

In [4]:
inlet = pn.pores('left')
outlet = pn.pores('right')
flow = op.algorithms.StokesFlow(network=pn, phase=phase)
flow.set_value_BC(pores=inlet, values=1)
flow.set_value_BC(pores=outlet, values=0)
flow.run()

{'pore.pressure': SteadyStateSolution([1., 1., 1., ..., 0., 0., 0.])}

### Calculate permeability

Calculate the permeability using Darcy's law. As pressure difference and viscosity were assumed to be 1, we have a simplified equation:

In [5]:
# NBVAL_IGNORE_OUTPUT
Q = flow.rate(pores=inlet, mode='group')[0]
A = op.topotools.get_domain_area(pn, inlets=inlet, outlets=outlet)
L = op.topotools.get_domain_length(pn, inlets=inlet, outlets=outlet)
# K = Q * L * mu / (A * Delta_P) # mu and Delta_P were assumed to be 1.
K = Q * L / A
print(f'The value of K is: {K/0.98e-12*1000:.2f} mD')

------------------------------------------------------------
    SOURCE     : openpnm.topotools._topotools.get_domain_area 
    TIME STAMP : 2022-07-13 12:46:51,362    
------------------------------------------------------------
------------------------------------------------------------
             boundary pores were not added 
    SOURCE     : openpnm.topotools._topotools.get_domain_length 
    TIME STAMP : 2022-07-13 12:46:51,372    
------------------------------------------------------------


The value of K is: 0.07 mD
